In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./airbnb_listing_train.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price
0,11602914,Luxury home 100m2+garden in center!,3123809,Marjolein,NaN,De Pijp - Rivierenbuurt,52.35368,4.90177,Entire home/apt,3,13,15-02-2020,0.27,1,0,220
1,13289321,East is where your home is!,10259430,Ana,NaN,Oud-Oost,52.35728,4.92052,Entire home/apt,4,14,19-06-2019,0.29,1,0,110
2,40779315,Amsterdam Central - LUXURY house in BUSY STREET,224969266,Mark,NaN,Centrum-West,52.37582,4.89850,Entire home/apt,2,9,16-03-2020,1.65,1,7,100
3,7820311,great appartment down town,693472,Ayden,NaN,Westerpark,52.37792,4.87300,Entire home/apt,3,42,17-02-2020,0.72,1,0,130
4,27346603,Private room with canal view in peaceful area,41888346,Alissa,NaN,Westerpark,52.38051,4.87493,Private room,2,89,26-02-2020,4.02,1,24,90


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12901 entries, 0 to 12900
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              12901 non-null  int64  
 1   name                            12879 non-null  object 
 2   host_id                         12901 non-null  int64  
 3   host_name                       12893 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   12901 non-null  object 
 6   latitude                        12901 non-null  float64
 7   longitude                       12901 non-null  float64
 8   room_type                       12901 non-null  object 
 9   minimum_nights                  12901 non-null  int64  
 10  number_of_reviews               12901 non-null  int64  
 11  last_review                     11305 non-null  object 
 12  reviews_per_month               

In [4]:
df.drop(columns=['neighbourhood_group'],inplace=True)

In [5]:
df.isnull().sum()

id                                   0
name                                22
host_id                              0
host_name                            8
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
last_review                       1596
reviews_per_month                 1596
calculated_host_listings_count       0
availability_365                     0
price                                0
dtype: int64

In [12]:
df.drop(list(df.loc[df.host_name.isna() == True].index),inplace=True)

In [13]:
df.isnull().sum()

id                                   0
name                                22
host_id                              0
host_name                            0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
last_review                       1596
reviews_per_month                 1596
calculated_host_listings_count       0
availability_365                     0
price                                0
dtype: int64

In [14]:
df[['host_id','host_name']].nunique()

host_id      11593
host_name     4536
dtype: int64

In [16]:
len(df.loc[(df.name.isna() == True)& (df.reviews_per_month.isna()==True) & (df.last_review.isna()==True)])

7

In [17]:
df.drop(list(df.loc[(df.name.isna() == True)& (df.reviews_per_month.isna()==True) & (df.last_review.isna()==True)].index),inplace=True)

In [18]:
df.nunique()

id                                12886
name                              12645
host_id                           11586
host_name                          4535
neighbourhood                        22
latitude                           5226
longitude                          7880
room_type                             4
minimum_nights                       56
number_of_reviews                   362
last_review                        1495
reviews_per_month                   609
calculated_host_listings_count       26
availability_365                    366
price                               400
dtype: int64

In [20]:
df.shape

(12886, 15)

In [28]:
df = df[['neighbourhood','room_type','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365','price']]

In [29]:
df.isnull().sum()

neighbourhood                        0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                 1589
calculated_host_listings_count       0
availability_365                     0
price                                0
dtype: int64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12886 entries, 0 to 12900
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood                   12886 non-null  object 
 1   room_type                       12886 non-null  object 
 2   minimum_nights                  12886 non-null  int64  
 3   number_of_reviews               12886 non-null  int64  
 4   reviews_per_month               11297 non-null  float64
 5   calculated_host_listings_count  12886 non-null  int64  
 6   availability_365                12886 non-null  int64  
 7   price                           12886 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 906.0+ KB


In [35]:
cat_columns = [col for col in df.select_dtypes(['object'])]
num_columns = list(set(df.columns) - set(cat_columns))
num_columns.remove('price')
print(cat_columns)
print(num_columns)

['neighbourhood', 'room_type']
['availability_365', 'reviews_per_month', 'minimum_nights', 'calculated_host_listings_count', 'number_of_reviews']


In [41]:
x_cols = [col for col in df.columns if col != 'price']
X = df[x_cols]
y = df['price']

In [43]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [44]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [46]:
OH_columns = [col for col in cat_columns if df[col].nunique()<=10]
LE_columns = list(set(cat_columns)-set(OH_columns))

In [47]:
num_transformer = SimpleImputer(missing_values=np.nan,strategy='mean')
OH_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(missing_values=np.nan,strategy='most_frequent')),
    ('onehot',OneHotEncoder())
])
LE_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(missing_values=np.nan,strategy='most_frequent')),
    ('le_enc',OrdinalEncoder())
])

In [48]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num_transformer',num_transformer,num_columns),
        ('OH_transformer',OH_transformer,OH_columns),
        ('LE_transformer',LE_transformer,LE_columns)
    ]
)

In [49]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor

In [50]:
models = {
    'LR':LinearRegression(),
    'Ri':Ridge(alpha=1.0),
    'La':Lasso(alpha=1.0),
    'knn':KNeighborsRegressor(n_neighbors=5)
}

In [51]:
from sklearn.metrics import mean_absolute_error

adj_r2= {}
mse = {}

for name,model in models.items():
    pipe = Pipeline(steps=[
        ('preprocessor',preprocessor),
        ('regressor',model)
    ])
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)

    adj_r2[name] = round(pipe.score(x_test,y_test),3)
    mse[name] = mean_absolute_error(y_pred,y_test)

In [52]:
adj_r2

{'LR': 0.047, 'Ri': 0.047, 'La': 0.046, 'knn': 0.144}

In [53]:
mse

{'LR': 72.38695285360102,
 'Ri': 72.38779958493643,
 'La': 72.7282547321907,
 'knn': 79.23135023279875}